In [1]:
import pandas as pd
from rdkit.Chem import PandasTools
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import SDMolSupplier
from rdkit import RDConfig
from rdkit.Geometry import Point3D
import numpy as np

In [2]:
# Input and output file path initialization

sdf_paths = ['..\\..\\..\\..\\7-Docking HD score migliori\\240731_Top10PercentSelection\\2-fredHD_corretto\\OK_C1_Top10Percent_dockHD.sdf\\OK_C1_Top10Percent_dockHD.sdf',
         '..\\..\\..\\..\\7-Docking HD score migliori\\240731_Top10PercentSelection\\2-fredHD_corretto\\OK_C2_Top10Percent_dockHD.sdf\\OK_C2_Top10Percent_dockHD.sdf',
         '..\\..\\..\\..\\7-Docking HD score migliori\\240731_Top10PercentSelection\\2-fredHD_corretto\\OK_C3_Top10Percent_dockHD.sdf\\OK_C3_Top10Percent_dockHD.sdf',
         '..\\..\\..\\..\\7-Docking HD score migliori\\240731_Top10PercentSelection\\2-fredHD_corretto\\OK_C4_Top10Percent_dockHD.sdf\\OK_C4_Top10Percent_dockHD.sdf']

score_paths = ['..\\..\\..\\..\\7-Docking HD score migliori\\240731_Top10PercentSelection\\2-fredHD_corretto\\OK_C1_Top10Percent_dockHD_score.txt',
         '..\\..\\..\\..\\7-Docking HD score migliori\\240731_Top10PercentSelection\\2-fredHD_corretto\\OK_C2_Top10Percent_dockHD_score.txt',
         '..\\..\\..\\..\\7-Docking HD score migliori\\240731_Top10PercentSelection\\2-fredHD_corretto\\OK_C3_Top10Percent_dockHD_score.txt',
         '..\\..\\..\\..\\7-Docking HD score migliori\\240731_Top10PercentSelection\\2-fredHD_corretto\\OK_C4_Top10Percent_dockHD_score.txt']

where_to_save = "..\\..\\..\\..\\8-Filtraggio\\pose_similarity\\"


# Pose similarity calculation and file update

for n in range(len(sdf_paths)):


    # Dataframes initialization

    # Load .sdf file as pandas dataframe
    sdf_frame = PandasTools.LoadSDF(sdf_paths[n],smilesName='SMILES',molColName='Molecule',includeFingerprints=True)

    # Load score file as pandas dataframe
    score_frame = pd.read_csv(score_paths[n], sep='\t')


    # Average conformer structure calculation

    # Group the dataframe into chunks of 10 molecules (same molecule, different conformers)
    grouped = sdf_frame.groupby(sdf_frame.index // 10)

    mean_RMSD_tot = []

    for group_number, group in grouped:
        positions = []
        RMSD_group = []

        # Loop through the 10 conformers and collect atomic positions
        for i in range(10):
            mol = group["Molecule"].iloc[i]  # Get the ith molecule (conformer)
            conf = mol.GetConformer()        # Get the conformer
            coord = np.array(conf.GetPositions())  # Get atomic positions as a numpy array
            positions.append(coord)

        # Calculate the average atomic positions across the 10 conformers
        average_pos = np.mean(positions, axis=0)

        # Now update each conformer's atom positions to the average position

        mol0 = group["Molecule"].iloc[0]  # Get the first molecule (conformer)

        # Create a deep copy of the molecule to avoid modifying the original
        mol2 = Chem.Mol(mol0)
        conf2 = mol2.GetConformer()  # Get the conformer for updating positions

        # Update each atom's position to the averaged position
        for m in range(conf2.GetNumAtoms()):
            x, y, z = average_pos[m]
            conf2.SetAtomPosition(m, Point3D(x, y, z))

        #
        for j in range(10):
            rmsd = AllChem.CalcRMS(group["Molecule"].iloc[j], mol2)
            RMSD_group.append(rmsd)

        mean_RMSD = np.mean(RMSD_group)
        mean_RMSD_tot.append(mean_RMSD)

    # Dataframe formatting

    # Save only the first pose (best score) for each compound, add average RMSD column
    sdf_frame2 = pd.DataFrame(sdf_frame.loc[sdf_frame["Pose Index"] == str(1)]) # Conformer sdf dataframe
    sdf_frame2["Average RMSD"] = mean_RMSD_tot

    score_frame2 = pd.DataFrame(score_frame.loc[score_frame["Pose"] == 0]) # Docking scores dataframe
    score_frame2["Average RMSD"] = mean_RMSD_tot


    # Saving

    # Save updated .sdf file
    #sdf_frame2 = sdf_frame2[['FRED Chemgauss4 score','CG3:Steric','CG3:Clash','CG3:ProDesolv','CG3:LigDesolv','CG3:LigDesolvHB','CG4:HB','Pose Index','ID','SMILES','Molecule', 'Average RMSD']]

    PandasTools.WriteSDF(sdf_frame2, where_to_save + "C" + str(n+1) + "_Top_dockHD_avrgRMSD.sdf",molColName='Molecule',
                         properties=['FRED Chemgauss4 score','CG3:Steric','CG3:Clash','CG3:ProDesolv','CG3:LigDesolv','CG3:LigDesolvHB','CG4:HB','Pose Index'], idName="ID")

    # Save updated score file
    score_frame2.to_csv(where_to_save + "C" + str(n+1) + "_Top_dockHD_score_avrgRMSD.txt", index=False)


KeyboardInterrupt: 

In [ ]:
# Input and output file path initialization

sdf_paths = ['..\\..\\..\\..\\7-Docking HD score migliori\\240731_Top10PercentSelection\\2-fredHD_corretto\\OK_C1_Top10Percent_dockHD.sdf\\OK_C1_Top10Percent_dockHD.sdf',
         '..\\..\\..\\..\\7-Docking HD score migliori\\240731_Top10PercentSelection\\2-fredHD_corretto\\OK_C2_Top10Percent_dockHD.sdf\\OK_C2_Top10Percent_dockHD.sdf',
         '..\\..\\..\\..\\7-Docking HD score migliori\\240731_Top10PercentSelection\\2-fredHD_corretto\\OK_C3_Top10Percent_dockHD.sdf\\OK_C3_Top10Percent_dockHD.sdf',
         '..\\..\\..\\..\\7-Docking HD score migliori\\240731_Top10PercentSelection\\2-fredHD_corretto\\OK_C4_Top10Percent_dockHD.sdf\\OK_C4_Top10Percent_dockHD.sdf']

score_paths = ['..\\..\\..\\..\\7-Docking HD score migliori\\240731_Top10PercentSelection\\2-fredHD_corretto\\OK_C1_Top10Percent_dockHD_score.txt',
         '..\\..\\..\\..\\7-Docking HD score migliori\\240731_Top10PercentSelection\\2-fredHD_corretto\\OK_C2_Top10Percent_dockHD_score.txt',
         '..\\..\\..\\..\\7-Docking HD score migliori\\240731_Top10PercentSelection\\2-fredHD_corretto\\OK_C3_Top10Percent_dockHD_score.txt',
         '..\\..\\..\\..\\7-Docking HD score migliori\\240731_Top10PercentSelection\\2-fredHD_corretto\\OK_C4_Top10Percent_dockHD_score.txt']

where_to_save = "..\\..\\..\\..\\8-Filtraggio\\pose_similarity\\"


# Pose similarity calculation and file update

for n in range(len(sdf_paths)):


    # Dataframes initialization

    # Load .sdf file as pandas dataframe
    sdf_supplier = SDMolSupplier(sdf_paths[n])

    # Load score file as pandas dataframe
    score_frame = pd.read_csv(score_paths[n], sep='\t')


    # Average conformer structure calculation

    # Group the dataframe into chunks of 10 molecules (same molecule, different conformers)
    grouped = sdf_frame.groupby(sdf_frame.index // 10)

    mean_RMSD_tot = []

    for group_number, group in grouped:
        positions = []
        RMSD_group = []

        # Loop through the 10 conformers and collect atomic positions
        for i in range(10):
            mol = group["Molecule"].iloc[i]  # Get the ith molecule (conformer)
            conf = mol.GetConformer()        # Get the conformer
            coord = np.array(conf.GetPositions())  # Get atomic positions as a numpy array
            positions.append(coord)

        # Calculate the average atomic positions across the 10 conformers
        average_pos = np.mean(positions, axis=0)

        # Now update each conformer's atom positions to the average position

        mol0 = group["Molecule"].iloc[0]  # Get the first molecule (conformer)

        # Create a deep copy of the molecule to avoid modifying the original
        mol2 = Chem.Mol(mol0)
        conf2 = mol2.GetConformer()  # Get the conformer for updating positions

        # Update each atom's position to the averaged position
        for m in range(conf2.GetNumAtoms()):
            x, y, z = average_pos[m]
            conf2.SetAtomPosition(m, Point3D(x, y, z))

        #
        for j in range(10):
            rmsd = AllChem.CalcRMS(group["Molecule"][j], mol2)
            RMSD_group.append(rmsd)

        mean_RMSD = mean(RMSD_group)
        mean_RMSD_tot.append(mean_RMSD)

    # Dataframe formatting

    # Save only the first pose (best score) for each compound, add average RMSD column
    sdf_frame2 = pd.DataFrame(sdf_frame.loc[sdf_frame["Pose Index"] == str(1)]) # Conformer sdf dataframe
    sdf_frame2["Average RMSD"] = mean_RMSD_tot

    score_frame2 = pd.DataFrame(score_frame.loc[score_frame["Pose"] == 0]) # Docking scores dataframe
    score_frame2["Average RMSD"] = mean_RMSD_tot


    # Saving

    # Save updated .sdf file
    #sdf_frame2 = sdf_frame2[['FRED Chemgauss4 score','CG3:Steric','CG3:Clash','CG3:ProDesolv','CG3:LigDesolv','CG3:LigDesolvHB','CG4:HB','Pose Index','ID','SMILES','Molecule', 'Average RMSD']]

    PandasTools.WriteSDF(sdf_frame2, where_to_save + "C" + str(n+1) + "_Top_dockHD_avrgRMSD.sdf",molColName='Molecule',
                         properties=['FRED Chemgauss4 score','CG3:Steric','CG3:Clash','CG3:ProDesolv','CG3:LigDesolv','CG3:LigDesolvHB','CG4:HB','Pose Index'], idName="ID")

    # Save updated score file
    score_frame2.to_csv(where_to_save + "C" + str(n+1) + "_Top_dockHD_score_avrgRMSD.txt", index=False)


rdkit.Chem.PandasTools.WriteSDF(df, out, molColName='ROMol', idName=None, properties=None, allNumeric=False, forceV3000=False)

Write an SD file for the molecules in the dataframe. Dataframe columns can be exported as SDF tags if specified in the “properties” list. “properties=list(df.columns)” would export all columns. The “allNumeric” flag allows to automatically include all numeric columns in the output. User has to make sure that correct data type is assigned to column. “idName” can be used to select a column to serve as molecule title. It can be set to “RowID” to use the dataframe row key as title.

In [ ]:
#Fare in modo che funzioni per tutti e 4 i recettori
#Selezionare solo le colonne desiderate
#Salvare in .sdf il file con RMSD medio
#Salvare in .sdf il file con tutte le pose, ciascuna con il suo RMSD
#Caricare il file degli score
#Mantenere solo le righe corrispondenti alla prima posa
#Aggiungere la colonna dell'RMSD medio